In [7]:
import asyncio
from typing import Sequence
from autogen_agentchat.agents import BaseChatAgent
from autogen_agentchat.base import Response
from autogen_agentchat.messages import BaseChatMessage, TextMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core import CancellationToken
import requests


In [8]:
class RAGAgent(BaseChatAgent):
    def __init__(self, name: str):
        super().__init__(name, description="用于执行 RAG 检索服务。")
        self._message_history: list[BaseChatMessage] = []

    @property
    def produced_message_types(self) -> Sequence[type[BaseChatMessage]]:
        return (TextMessage,)

    async def on_messages(self, messages: Sequence[BaseChatMessage], cancellation_token: CancellationToken) -> Response:
        self._message_history.extend(messages)
        query = self._message_history[-1].content
        
        simulated_response = {"income_rmb": 1845.04}  
        #requests.post("http://your-rag-service/api", json={"query": query})
        result = f"腾讯 2025 年第二季度收入：{simulated_response['income_rmb']} 亿元人民币。"
        response_message = TextMessage(content=result, source=self.name)
        self._message_history.append(response_message)
        return Response(chat_message=response_message)

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        self._message_history.clear()

In [9]:
class MCPAgent(BaseChatAgent):
    def __init__(self, name: str):
        super().__init__(name, description="执行 MCP 汇率工具，进行货币换算。")
        self._message_history: list[BaseChatMessage] = []

    async def on_messages(self, messages: Sequence[BaseChatMessage], cancellation_token: CancellationToken) -> Response:
        self._message_history.extend(messages)
        input_data = self._message_history[-1].content  # 获取前一步 RAG 输出
        # 解析收入（假设从输入中提取数字）
        import re
        income_rmb = float(re.search(r'(\d+\.?\d*) 亿元人民币', input_data).group(1))
        # 模拟调用 MCP 汇率 API（替换为您的实际端点）
        # 假设汇率：1 RMB = 0.1202 EUR（2025 年 10 月数据）
        exchange_rate = 0.1202  
        # response = requests.get("http://your-mcp-service/api/exchange?from=CNY&to=EUR")
        income_eur = income_rmb * exchange_rate
        result = f"换算后欧元收入：{income_eur:.2f} 亿元欧元（汇率 1 RMB ≈ {exchange_rate} EUR）。"
        response_message = TextMessage(content=result, source=self.name)
        self._message_history.append(response_message)
        return Response(chat_message=response_message)

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        self._message_history.clear()

In [10]:
class IntegratorAgent(BaseChatAgent):
    def __init__(self, name: str):
        super().__init__(name, description="整合所有结果，生成最终答案。")
        self._message_history: list[BaseChatMessage] = []

    async def on_messages(self, messages: Sequence[BaseChatMessage], cancellation_token: CancellationToken) -> Response:
        self._message_history.extend(messages)
        # 收集历史结果
        rag_result = [msg.content for msg in self._message_history if msg.source == "rag_agent"][-1]
        mcp_result = [msg.content for msg in self._message_history if msg.source == "mcp_agent"][-1]
        final_answer = f"查询结果：{rag_result} {mcp_result}"
        response_message = TextMessage(content=final_answer, source=self.name)
        self._message_history.append(response_message)
        return Response(chat_message=response_message)

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        self._message_history.clear()

In [12]:
async def run_demo(query: str):
    # 初始化代理
    rag_agent = RAGAgent("rag_agent")
    mcp_agent = MCPAgent("mcp_agent")
    integrator_agent = IntegratorAgent("integrator_agent")

    # 配置 SelectorGroupChat（LLM 规划顺序）
    selector_group_chat = SelectorGroupChat(
        agents=[rag_agent, mcp_agent, integrator_agent],
        model_client=OpenAIChatCompletionClient(model="gpt-4o"),  # 替换为您的 LLM 配置
        termination_condition=MaxMessageTermination(5),  # 最大消息终止
        allow_repeated_speaker=True,
        selector_prompt=(
            "可用代理：{roles}\n其描述：{participants}\n"
            "基于查询 '{query}'，选择最合适的代理，仅返回代理名称。"
        ),
    )

    # 启动任务
    task = [TextMessage(content=query, source="user")]
    stream = selector_group_chat.run_stream(task=task)
    final_output = ""
    async for message in stream:
        if isinstance(message, TextMessage):
            final_output += message.content + "\n"
    return final_output

In [13]:
query = "腾讯2025年财报换算成欧元收入是多少？"
result = asyncio.run(run_demo(query))
print("最终输出：\n", result)

RuntimeError: asyncio.run() cannot be called from a running event loop